# import package and data

In [1]:
import pandas as pd
import os
import configparser
from neo4j import GraphDatabase
import textwrap

# Set the display format for float values, to avoid using scientific notation for big number
pd.options.display.float_format = '{:.0f}'.format

# Connect to neo4j

In [2]:
# Using an ini file for credentials, otherwise providing defaults
HOST = 'neo4j://localhost'
DATABASE = 'neo4j'
PASSWORD = 'password'

NEO4J_CONF_FILE = 'neo4j.ini'

if NEO4J_CONF_FILE is not None and os.path.exists(NEO4J_CONF_FILE):
    config = configparser.RawConfigParser()
    config.read(NEO4J_CONF_FILE)
    HOST = config['NEO4J']['HOST']
    DATABASE = config['NEO4J']['DATABASE']
    PASSWORD = config['NEO4J']['PASSWORD']
    print('Using custom database properties')
else:
    print('Could not find database properties file, using defaults')

Using custom database properties


In [3]:
# create neo4j Python driver
driver = GraphDatabase.driver(HOST, auth=(DATABASE, PASSWORD))

In [4]:
# helper function
def run(driver, query, params=None):
    with driver.session() as session:
        if params is not None:
            return [r for r in session.run(query, params)]
        else:
            return [r for r in session.run(query)]

## set unique constraints

In [5]:
run(driver,'CREATE CONSTRAINT user_id_unique IF NOT EXISTS FOR (user:User) REQUIRE user.id IS UNIQUE')

run(driver,'CREATE CONSTRAINT origin_id_unique IF NOT EXISTS FOR (origin:Origin) REQUIRE origin.id IS UNIQUE')

run(driver,'CREATE CONSTRAINT poi_id_unique IF NOT EXISTS FOR (poi:Poi) REQUIRE poi.id IS UNIQUE')

run(driver,'CREATE CONSTRAINT category_id_unique IF NOT EXISTS FOR (category:Category) REQUIRE category.id IS UNIQUE')

run(driver,'CREATE CONSTRAINT region_id_unique IF NOT EXISTS FOR (region:Region) REQUIRE region.id IS UNIQUE')

run(driver,'CREATE CONSTRAINT review_id_unique IF NOT EXISTS FOR (review:Review) REQUIRE review.id IS UNIQUE')

[]

hosted .csv on Github,
below are the path to find .csv files

if using dropbox to host the .csv file
steps:
1. make sure the share permission is "Anyone with the link can view"
2. download the file, go to browser download center
3. right click on the download of .csv, copy link address, replace the url below

In [6]:
# import .csv files are hostsed on Github for easier access

url_node_category = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_node_category.csv'

url_node_origin = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_node_origin.csv'

url_node_poi = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_node_poi.csv'

url_node_region = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_node_region.csv'

url_node_review_1 = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_node_review_1.csv'

url_node_review_2 = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_node_review_2.csv'

url_node_user = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_node_user.csv'

url_poi_belongsto_category = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_poi_belongsto_category.csv'

url_poi_locatedat_region = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_poi_locatedat_region.csv'

url_user_from_origin = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_user_from_origin.csv'

url_user_reviewed_poi = 'https://raw.githubusercontent.com/xiong-ying/neo4j-import/main/df_user_reviewed_poi.csv'


# directory = "neo4j_import"

## Load Nodes

In [7]:
# df_node_category
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MERGE(category:Category {id: toInteger(row.id), name: row.name})
    RETURN count(category)
    """),
    params = {'file': url_node_category}
)

[<Record count(category)=42>]

In [8]:
# df_node_origin
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MERGE(origin:Origin {id: toInteger(row.id), name: row.name})
    RETURN count(origin)
    """),
    params = {'file': url_node_origin}
)

[<Record count(origin)=9403>]

In [9]:
#df_node_poi
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MERGE (poi:Poi {id: toInteger(row.id)})
    ON CREATE SET
        poi.name = coalesce(row.name, ''),
        poi.description = coalesce(row.description, ''),
        poi.url = coalesce(row.url, ''),
        poi.openingHours = coalesce(row.openingHours, ''),
        poi.duration = coalesce(row.duration, ''),
        poi.price = toFloat(coalesce(row.price, '0.0')),
        poi.address = coalesce(row.address, ''),
        poi.avgRating = toFloat(coalesce(row.avgRating, '0.0')),
        poi.numReviews = toInteger(coalesce(row.numReviews, '0')),
        poi.numReviews_5 = toInteger(coalesce(row.numReviews_5, '0')),
        poi.numReviews_4 = toInteger(coalesce(row.numReviews_4, '0')),
        poi.numReviews_3 = toInteger(coalesce(row.numReviews_3, '0')),
        poi.numReviews_2 = toInteger(coalesce(row.numReviews_2, '0')),
        poi.numReviews_1 = toInteger(coalesce(row.numReviews_1, '0'))
    RETURN count(poi)
    """),
    params = {'file': url_node_poi}
)


[<Record count(poi)=69>]

In [10]:
# df_node_region
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MERGE(region:Region {id: toInteger(row.id), name: row.name})
    RETURN count(region)
    """),
    params = {'file': url_node_region}
)

[<Record count(region)=29>]

In [11]:
#df_node_review_1
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MERGE (review:Review {id: toInteger(row.id)})
    ON CREATE SET
        review.title = coalesce(row.title, ''),
        review.date = date(coalesce(row.date, '')),
				review.rating = toFloat(coalesce(row.rating, '0.0')),
				review.content = coalesce(row.content, '')
    RETURN count(review)
    """),
    params = {'file': url_node_review_1}
)

[<Record count(review)=45227>]

In [12]:
#df_node_review_2
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MERGE (review:Review {id: toInteger(row.id)})
    ON CREATE SET
        review.title = coalesce(row.title, ''),
        review.date = date(coalesce(row.date, '')),
				review.rating = toFloat(coalesce(row.rating, '0.0')),
				review.content = coalesce(row.content, '')
    RETURN count(review)
    """),
    params = {'file': url_node_review_2}
)

[<Record count(review)=45228>]

In [13]:
# df_node_user
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MERGE(user:User {id: toInteger(row.id), name: row.name})
    RETURN count(user)
    """),
    params = {'file': url_node_user}
)

[<Record count(user)=58656>]

# Load Relationship

In [14]:
# df_poi_belongsto_category
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MATCH (poi:Poi {id: toInteger(row.poi_id)})
    MATCH (category:Category {id: toInteger(row.category_id)})
    MERGE (poi)-[r:BELONGS_TO]->(category)
    RETURN count(r) AS BELONGS_TO_count
    """),
    params = {'file': url_poi_belongsto_category}
)

[<Record BELONGS_TO_count=95>]

In [15]:
# df_poi_locatedat_region
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MATCH (poi:Poi {id: toInteger(row.poi_id)})
    MATCH (region:Region {id: toInteger(row.region_id)})
    MERGE (poi)-[r:LOCATED_AT]->(region)
    RETURN count(r) AS LOCATED_AT_count
    """),
    params = {'file': url_poi_locatedat_region}
)

[<Record LOCATED_AT_count=65>]

In [16]:
# df_user_from_origin
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    MATCH (user:User {id: toInteger(row.user_id)})
    MATCH (origin:Origin {id: toInteger(row.origin_id)})
    MERGE (user)-[r:FROM]->(origin)
    RETURN count(r) AS FROM_count
    """),
    params = {'file': url_user_from_origin}
)

[<Record FROM_count=51278>]

In [17]:
# df_user_reviewed_poi
run(driver, textwrap.dedent("""\
    LOAD CSV WITH HEADERS FROM $file AS row
    CALL{
        WITH row
        MATCH (user:User {id: toInteger(row.user_id)})
        MATCH (review:Review {id: toInteger(row.review_id)})
        MATCH (poi:Poi {id: toInteger(row.poi_id)})
        MERGE (user)-[w:WROTE]->(review)
        MERGE (review)-[rated:RATED]->(poi)
        MERGE (user)-[reviewed:REVIEWED ]->(poi)
        ON CREATE SET reviewed.rating = review.rating
        RETURN count(w) AS WROTE_count, count(rated) AS RATED_count, count(reviewed) AS REVIEWED_count
    } IN TRANSACTIONS
    RETURN SUM(WROTE_count) AS total_WROTE_count, SUM(RATED_count) AS total_RATED_count, SUM(REVIEWED_count) AS total_REVIEWED_count
    """),
    params = {'file': url_user_reviewed_poi}
)

[<Record total_WROTE_count=90455 total_RATED_count=90455 total_REVIEWED_count=90455>]

# Close the driver

In [18]:
driver.close()